In [2]:
%load_ext autoreload
%autoreload 0

import os, json
from draftfast import rules
from draftfast.optimize import run
from draftfast.orm import Player
from draftfast.csv_parse import salary_download
from draftfast.settings import OptimizerSettings, CustomRule, PlayerPoolSettings
from draftfast.lineup_constraints import LineupConstraints
import pandas as pd
import numpy as np
from spread import get_spread_data, get_current_rankings
from collections import defaultdict

WEEK = 14
SALARY_FILE = f"./history/week{WEEK}.csv"
MIN_SALARY = 0

# http://rotoguru1.com/cgi-bin/fyday.pl?week=13&game=fd&scsv=1
ACTIVE_FILE = f"./history/optimal{WEEK}.csv"

print('ready')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ready


In [3]:
df = pd.read_csv(SALARY_FILE, delimiter=";")
rename_map = {
    'Pos': 'Position',
    'FD points': 'FPPG',
    'Name': 'Nickname',
    'FD salary': 'Salary'
}
df = df.rename(columns=rename_map)[['Nickname', 'Position', 'FPPG', 'Salary']]
df = df[~df['Salary'].isna()]
df['Position'] = df['Position'].apply(lambda x: 'D' if x == 'Def' else x)

print(df.columns.values)

df.to_csv(ACTIVE_FILE)
df.describe()

['Nickname' 'Position' 'FPPG' 'Salary']


,FPPG,Salary
count,388.000000,388.000000
mean,6.292113,5320.103093
std,7.562498,1276.323041
min,-3.000000,0.000000
25%,0.000000,4500.000000
50%,3.100000,4800.000000
75%,10.450000,6025.000000
max,36.220000,9200.000000


In [4]:
players = salary_download.generate_players_from_csvs(salary_file_location=ACTIVE_FILE, game=rules.FAN_DUEL)
print(players[0].__dict__)

{'pos': 'QB', 'name': 'Allen, Josh', 'cost': 8800.0, 'team': None, 'matchup': None, 'proj': 36.22, 'average_score': 36.22, 'projected_ownership_pct': 0, 'lineup_count': 0, 'marked': None, 'lock': False, 'position_lock': False, 'ban': False, 'position_ban': False, 'multi_position': False, 'possible_positions': 'QB', 'kv_store': OrderedDict([('', '0'), ('Nickname', 'Allen, Josh'), ('Position', 'QB'), ('FPPG', '36.22'), ('Salary', '8800.0')])}


In [5]:
def get_nfl_positions():
    return [
        ['QB', 1, 1],
        ['RB', 2, 3],
        ['WR', 3, 4],
        ['TE', 1, 2],
        ['D', 1, 1]
    ]     
ACTIVE_RULE_SET = rules.FD_NFL_RULE_SET
# Overrides (position limits, salary, roster size, positions, etc.
ACTIVE_RULE_SET.salary_max = 60000
ACTIVE_RULE_SET.defensive_positions = ['D', 'DEF']
ACTIVE_RULE_SET.offensive_positions = ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX']
ACTIVE_RULE_SET.position_limits = get_nfl_positions()
ACTIVE_RULE_SET.max_players_per_team = 9
ACTIVE_RULE_SET.roster_size = 9
print(ACTIVE_RULE_SET.__dict__)

{'site': 'FAN_DUEL', 'league': 'NFL', 'roster_size': 9, 'position_limits': [['QB', 1, 1], ['RB', 2, 3], ['WR', 3, 4], ['TE', 1, 2], ['D', 1, 1]], 'general_position_limits': [], 'salary_min': 0, 'salary_max': 60000, 'offensive_positions': ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX'], 'defensive_positions': ['D', 'DEF'], 'game_type': 'classic', 'max_players_per_team': 9, 'position_per_team_rules': None, 'min_teams': None}


In [6]:
set(df['Position'])

{'D', 'QB', 'RB', 'TE', 'WR'}

In [7]:
print(f"Optional Week {WEEK} Roster")
roster = run(
        rule_set=ACTIVE_RULE_SET,
        player_pool=players,
        verbose=False
    )
print(roster)

Optional Week 14 Roster
+----------+-----------------+------+---------+--------+------------+----------+--------+
| Position | Player          | Team | Matchup | Salary | Projection | vs. Avg. | Locked |
+----------+-----------------+------+---------+--------+------------+----------+--------+
| QB       | Allen, Josh     | None | None    |  8,800 |      36.22 |     0.00 |        |
| RB       | Cook, Dalvin    | None | None    |  8,200 |       34.7 |     0.00 |        |
| RB       | Conner, James   | None | None    |  7,200 |       29.0 |     0.00 |        |
| RB       | Penny, Rashaad  | None | None    |  5,300 |       26.3 |     0.00 |        |
| WR       | Adams, Davante  | None | None    |  8,400 |       29.1 |     0.00 |        |
| WR       | Renfrow, Hunter | None | None    |  6,700 |       22.2 |     0.00 |        |
| WR       | Sims, Cam       | None | None    |  4,600 |       14.4 |     0.00 |        |
| TE       | Kittle, George  | None | None    |  7,100 |       27.6 |     0.